# Quote Generation/Retrieval



In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os
import openai

# import utils
import configparser
from pprint import pprint
from llama_index import SimpleDirectoryReader


import os
import openai


import jupyter_black

jupyter_black.load()
# Access values from the sections

import sys

sys.path.append("../app")
import conn_utils

OPENAI_API_KEY = conn_utils.get_open_ai_key("../config.ini")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
directory = "index_store"

In [11]:
!ls ./../data/

Amours.txt               Heroides.txt             MetamorphosesVIII_XV.txt
Fasti.txt                MetamorphosesI_VII.txt   RemediaAmoris.txt


In [14]:
f1 = "./../data/RemediaAmoris.txt"
f3 = "./../data/Heroides.txt"
f4 = "./../data/Amours.txt"
f5 = "./../data/Fasti.txt"
f6 = "./../data/MetamorphosesI_VII.txt"
f7 = "./../data/MetamorphosesVIII_XV.txt"
docs = [f1, f3, f4, f5, f6, f7]
documents = SimpleDirectoryReader(input_files=docs).load_data()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
texts = text_splitter.create_documents(docs)
directory = "index_store"
vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
vector_index.save_local(directory)

vector_index = FAISS.load_local("index_store", OpenAIEmbeddings())
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [51]:
qa_interface = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

response = qa_interface(
    """
I am a big fan of ovid. 
Please recommend memorable quotes to me.
"""
)

print(response["result"])

Certainly! Here are some memorable quotes from Ovid's works:

1. "Love is a kind of warfare." - Amores

2. "Fortune favors the bold." - Metamorphoses

3. "One man's meat is another man's poison." - Metamorphoses

4. "The cause is hidden, but the effect is visible to all." - Metamorphoses

5. "Wherever there is a human being, there is an opportunity for a kindness." - Metamorphoses

6. "If you want to be loved, be lovable." - Remedia Amoris

7. "The enemy you know is better than the one you don't." - Heroides

8. "Time flies, death urges, heaven's revolving wheel turns swift, and nothing suffers long." - Fasti

I hope you find these quotes memorable and enjoyable!


### 2 Issues with Index retrieved/Generated Quotes:
The following should be conditional 1) That the quote generated has confidence that the quote was written by Ovid then 2) That the quote is not a variation of a quote already in DB.

#### Authorship authentication


In [61]:



 from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """Task: On a scale of 0 to 1, with 0 indicating low confidence
and 1 indicating high confidence, please provide a general
assessment of the likelihood that given text 
written by the same author as the provided reference. Your answer should reflect a
moderate level of strictness in scoring. Here are some
relevant variables to this problem.
1. punctuation style(e.g. hyphen, brackets, colon, comma,
parenthesis, quotation mark)
2. special characters style, capitalization style(e.g.
Continuous capitalization, capitalizing certain words)
3. acronyms and abbreviations(e.g. Usage of acronyms
such as OMG, Abbreviations without punctuation marks
such as Mr Rochester vs. Mr. Rochester,Unusual
abbreviations such as def vs. definitely)
4. writing style
5. expressions and Idioms
6. tone and mood
7. sentence structure
8. any other relevant aspect
First step: Understand the problem, extracting relevant
variables and devise a plan to solve the problem. Then,
carry out the plan and solve the problem step by step.
9. One (or both) of the texts is written by the famous Latin author "Ovid"
Finally, show the confidence score.

The following are all quotes by Ovid for reference:
'Love is a thing full of anxious fears.',
 'Now are fields of corn where Troy once stood.',
 "We're slow to believe what wounds us.",
 "The end proves the acts (were done), or the result is a test of the actions; Ovid's line 85 full translation: “The event proves well the wisdom of her [Phyllis'] course.”",
 "Let him who loves, where love success may find, Spread all his sails before the prosp'rous wind.",
 'Resist beginnings; the remedy comes too late when the disease has gained strength by long delays.',
 "Love yields to business. If you seek a way out of love, be busy; you'll be safe then.",
 'The gods behold all righteous actions.',
 'There is a god within us.',
 'The mind, conscious of rectitude, laughed to scorn the falsehood of report.',
 'Every lover is a soldier.',
 'Let the man who does not wish to be idle fall in love!',
 'Far away be that fate!',
 'They bear punishment with equanimity who have earned it.',
 "We take no pleasure in permitted joys. But what's forbidden is more keenly sought.",
 'Who is allowed to sin, sins less.' """
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
#chain.invoke({"text": ""})

['Based on the provided quotes by Ovid',
 'it is difficult to assess the likelihood that the given text is written by the same author. The quotes by Ovid showcase a variety of writing styles',
 'expressions',
 'and tones',
 'making it challenging to determine a consistent pattern. Additionally',
 'the given text does not provide enough information to make a confident judgement. Therefore',
 'the confidence score for this assessment is 0.5.']

In [99]:
import re

non_ovid_generated_quotes = [
    "One man's meat is another man's poison.",
    "Fortune favors the bold.",
    "Wherever there is a human being, there is an opportunity for a kindness.",
    "Love is a kind of warfare.",
    "One man's meat is another man's poison.",
]


# Define the regex pattern
pattern = r"(\d+\.\d+)"


scores = []
strings = []
for q in non_ovid_generated_quotes:
    r = chain.invoke({"text": q})
    r = " ".join(r)
    strings.append(r)
    match = re.search(pattern, r)
    score = match[0] if match else -1
    scores.append(score)
    print(r)

pd.DataFrame(
    data={
        "questionable_quote": non_ovid_generated_quotes,
        "authorship_match_score": scores,
        "is_original": [0, 0, 0, 1, 0],
    }
)

Confidence Score: 0
On a scale of 0 to 1 I would assess the likelihood of the given text being written by the same author as Ovid as 0.5. While the quote does have a similar tone of encouragement and wisdom it lacks the specific style and language that is characteristic of Ovid's writing.
Confidence score: 0.3
Confidence Score: 0.8

Based on the provided quote "Love is a kind of warfare," the writing style and tone are similar to that of Ovid. The use of figurative language and comparison between love and warfare is in line with Ovid's style. The sentence structure is also similar with a concise and impactful statement. However without further context or comparison to other texts by Ovid it is difficult to determine with high confidence if the text is written by the same author.
0.3


,questionable_quote,authorship_match_score,is_original
0,One man's meat is another man's poison.,-1,0
1,Fortune favors the bold.,0.5,0
2,"Wherever there is a human being, there is an o...",0.3,0
3,Love is a kind of warfare.,0.8,1
4,One man's meat is another man's poison.,0.3,0


In [52]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

# Read in stored quotes
df = pd.read_json("./../ovid_quotes.json")

input_string = response["result"]


import re

# Extract lines starting with a number and strip the number
lines_with_stripped_numbers = [
    re.sub(r"^\d+\.\s*", "", line.strip())
    for line in input_string.splitlines()
    if re.match(r"^\d+\.", line)
]

# Print the extracted lines
for line in lines_with_stripped_numbers:
    print(line)

"Love is a kind of warfare." - Amores
"Fortune favors the bold." - Metamorphoses
"One man's meat is another man's poison." - Metamorphoses
"The cause is hidden, but the effect is visible to all." - Metamorphoses
"Wherever there is a human being, there is an opportunity for a kindness." - Metamorphoses
"If you want to be loved, be lovable." - Remedia Amoris
"The enemy you know is better than the one you don't." - Heroides
"Time flies, death urges, heaven's revolving wheel turns swift, and nothing suffers long." - Fasti


In [53]:
from itertools import product

# Compare to quotes
generated_quotes = lines_with_stripped_numbers

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Process each quote and calculate similarity scores
similarity_data = []

for generated_quote, actual_quote in product(generated_quotes, df["Quote"]):
    # Process the quotes with spaCy
    doc_generated = nlp(generated_quote)
    doc_actual = nlp(actual_quote)

    # Calculate similarity score
    similarity_score = doc_generated.similarity(doc_actual)

    # Append data to the list
    similarity_data.append(
        {
            "generated_quotes": generated_quote,
            "actual_quotes": actual_quote,
            "similarity_score": similarity_score,
        }
    )

# Create a DataFrame from the similarity data
similarity_df = pd.DataFrame(similarity_data)

/var/folders/qq/gdppvj912kv3ds7_xnf0q_fc0000gn/T/ipykernel_19754/867367726.py:18: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = doc_generated.similarity(doc_actual)


In [55]:
df["Quote"].to_list()

['Love is a thing full of anxious fears.',
 'Now are fields of corn where Troy once stood.',
 "We're slow to believe what wounds us.",
 "The end proves the acts (were done), or the result is a test of the actions; Ovid's line 85 full translation: “The event proves well the wisdom of her [Phyllis'] course.”",
 "Let him who loves, where love success may find, Spread all his sails before the prosp'rous wind.",
 'Resist beginnings; the remedy comes too late when the disease has gained strength by long delays.',
 "Love yields to business. If you seek a way out of love, be busy; you'll be safe then.",
 'The gods behold all righteous actions.',
 'There is a god within us.',
 'The mind, conscious of rectitude, laughed to scorn the falsehood of report.',
 'Every lover is a soldier.',
 'Let the man who does not wish to be idle fall in love!',
 'Far away be that fate!',
 'They bear punishment with equanimity who have earned it.',
 "We take no pleasure in permitted joys. But what's forbidden is mo

In [54]:
# Print the resulting DataFrame
display(similarity_df.sort_values("similarity_score"))

,generated_quotes,actual_quotes,similarity_score
123,"""Time flies, death urges, heaven's revolving w...",Let the man who does not wish to be idle fall ...,0.017135
27,"""Fortune favors the bold."" - Metamorphoses",Let the man who does not wish to be idle fall ...,0.068201
77,"""Wherever there is a human being, there is an ...",They bear punishment with equanimity who have ...,0.082028
66,"""Wherever there is a human being, there is an ...",We're slow to believe what wounds us.,0.108871
34,"""One man's meat is another man's poison."" - Me...",We're slow to believe what wounds us.,0.122052
...,...,...,...
115,"""Time flies, death urges, heaven's revolving w...","The end proves the acts (were done), or the re...",0.597987
35,"""One man's meat is another man's poison."" - Me...","The end proves the acts (were done), or the re...",0.606245
6,"""Love is a kind of warfare."" - Amores",Love yields to business. If you seek a way out...,0.609010
51,"""The cause is hidden, but the effect is visibl...","The end proves the acts (were done), or the re...",0.624752


In [ ]:
# For sanity-checking on how well a similarity_score does for retrieving a "memorable quote"

In [50]:
print("highest score comparison:")
idx = 54
print("GENERATED:", similarity_df.iloc[54]["generated_quotes"])
print("ACTUAL:", similarity_df.iloc[54]["actual_quotes"])

highest score comparison:
GENERATED: "Let others praise ancient times; I am glad I was born in these." - From Amores
ACTUAL: Love yields to business. If you seek a way out of love, be busy; you'll be safe then.


### Storage of Additional Quotes